In [ ]:
#| default_exp estacoes
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path


In [ ]:

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Anatel

> Este módulo consolida as bases da Anatel e realiza pós-processamento dos dados obtidos.

In [ ]:
#| export
import urllib.request
from typing import List
from zipfile import ZipFile

import geopandas as gpd
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.foundation import L
from fastcore.parallel import parallel
from fastcore.xtras import Path

from extracao.constants import COLS_SRD, IBGE_MUNICIPIOS, IBGE_POLIGONO, MALHA_IBGE
from extracao.datasources.aeronautica import Aero
from extracao.datasources.base import Base
from extracao.datasources.mosaico import MONGO_URI
from extracao.datasources.sitarweb import SQLSERVER_PARAMS, Radcom, Stel
from extracao.datasources.smp import SMP
from extracao.datasources.srd import SRD
from extracao.datasources.telecom import Telecom
from extracao.format import merge_on_frequency

In [ ]:
#| export
load_dotenv(find_dotenv())

True

## Base Consolidada ANATEL

`<<<<<<< HEAD`

In [ ]:
# |export
class Estacoes(Base):
    """Classe auxiliar para agregar os dados originários da Anatel"""

    def __init__(
        self,
        sql_params: dict = SQLSERVER_PARAMS,
        mongo_uri: str = MONGO_URI,
        limit: int = 0,
        parallel: bool = True,
    ):
        self.sql_params = sql_params
        self.mongo_uri = mongo_uri
        self.limit = limit
        self.parallel = parallel
        self.init_data_sources()

    @property
    def columns(self):
        return COLS_SRD

    def build_from_sources(self) -> pd.DataFrame:
        return self._format([s.df() for s in self.sources.values()])

    @property
    def stem(self):
        return "anatel"

    @staticmethod
    def _update_source(class_instance):
        class_instance.update()
        class_instance.save()
        return class_instance

    def init_data_sources(self):
        self.sources = {
            "telecom": Telecom(self.mongo_uri, self.limit),
            "smp": SMP(self.mongo_uri, self.limit),
            "srd": SRD(self.mongo_uri),
            "stel": Stel(self.sql_params),
            "radcom": Radcom(self.sql_params),
            "aero": Aero(),
        }

    def extraction(self) -> L:
        if self.parallel:
            sources = parallel(
                Estacoes._update_source,
                self.sources.values(),
                n_workers=len(self.sources),
                progress=True,
            )
        else:
            sources = L(self._update_source(s) for s in self.sources.values())
        return sources.attrgot("df")

    @staticmethod
    def verify_shapefile_folder():
        # Convert the file paths to Path objects
        shapefile_path = Path(IBGE_POLIGONO)
        parent_folder = shapefile_path.parent
        parent_folder.mkdir(exist_ok=True, parents=True)
        zip_file_path = parent_folder.with_suffix(".zip")

        # Check if all required files exist
        required_files = L(".cpg", ".dbf", ".prj", ".shx").map(
            shapefile_path.with_suffix
        )
        if not all(required_files.map(Path.is_file)):
            # shutil.rmtree(str(shapefile_path.parent), ignore_errors=True)
            parent_folder.ls().map(Path.unlink)
            # Download and unzip the zipped folder
            urllib.request.urlretrieve(MALHA_IBGE, zip_file_path)
            with ZipFile(zip_file_path, "r") as zip_ref:
                zip_ref.extractall(parent_folder)
            zip_file_path.unlink()

    def fill_nan_coordinates(
        self,
        df: pd.DataFrame,  # DataFrame com os dados da Anatel
    ) -> pd.DataFrame:  # DataFrame com as coordenadas validadas na base do IBGE
        """Valida as coordenadas consultado a Base Corporativa do IBGE, excluindo o que já está no cache na versão anterior"""

        municipios = pd.read_csv(
            IBGE_MUNICIPIOS,
            usecols=["Código_Município", "Latitude", "Longitude"],
            dtype="string[pyarrow]",
            dtype_backend="pyarrow",
        )

        df = pd.merge(df, municipios, on="Código_Município", how="left", copy=False)

        null_coords = df.Latitude_x.isna() | df.Longitude_x.isna()

        df.loc[null_coords, ["Latitude_x", "Longitude_x"]] = df.loc[
            null_coords, ["Latitude_y", "Longitude_y"]
        ]

        log = """[("Colunas", ["Latitude", "Longitude"]),
		           ("Processamento", "Coordenadas Ausentes. Inserido coordenadas do Município")]"""
        df = self.register_log(df, log, null_coords)

        df.rename(
            columns={
                "Latitude_x": "Latitude",
                "Longitude_x": "Longitude",
                "Latitude_y": "Latitude_ibge",
                "Longitude_y": "Longitude_ibge",
            },
            inplace=True,
        )

        return df

    def intersect_coordinates_on_poligon(
        self, df: pd.DataFrame, check_municipio: bool = True
    ):
        for column in ["Latitude", "Longitude"]:
            df[column] = pd.to_numeric(df[column], errors="coerce").astype("float")
        regions = gpd.read_file(IBGE_POLIGONO)

        # Convert pandas dataframe to geopandas df with geometry point given coordinates
        gdf_points = gpd.GeoDataFrame(
            df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude)
        )

        # Set the same coordinate reference system (CRS) as the regions shapefile
        gdf_points.crs = regions.crs

        # Spatial join points to the regions
        gdf = gpd.sjoin(gdf_points, regions, how="inner", predicate="within")

        if check_municipio:
            # Check correctness of Coordinates
            check_coords = gdf.Código_Município != gdf.CD_MUN

            log = """[("Colunas", ["Código_Município", "Município", "UF"]),
				  	 ("Processamento", "Informações substituídas  pela localização correta das coordenadas.")		      
				  """
			self.register_log(gdf, log, check_coords)

			gdf.drop(['Código_Município', 'Município', 'UF'], axis=1, inplace=True)

		gdf.rename(
			columns={
				'CD_MUN': 'Código_Município',
				'NM_MUN': 'Município',
				'SIGLA_UF': 'UF',
			},
			inplace=True,
		)

		return gdf

	def validate_coordinates(self, df: pd.DataFrame, check_municipio: bool = True) -> pd.DataFrame:
		"""
		Validates the coordinates in the given DataFrame.

		Args:
		        df: The DataFrame containing the coordinates to be validated.
		        check_municipio: A boolean indicating whether to check the municipality information (default: True).

		Returns:
		        pd.DataFrame: The DataFrame with validated coordinates.

		Raises:
		        None
		"""
		self.verify_shapefile_folder()
		if check_municipio:
			df = self.fill_nan_coordinates(df)
		return self.intersect_coordinates_on_poligon(df, check_municipio)

	@staticmethod
	def _simplify_sources(df):
		df['Fonte'] = df['Fonte'].str.replace(
			'ICAO-CANALIZACAO-VOR/ILS/DME | AISWEB-CANALIZACAO-VOR/ILS/DME',
			'CANALIZACAO-VOR/ILS/DME',
		)
		df['Fonte'] = df['Fonte'].str.replace(
			r'(ICAO-)?(AISWEB-)?CANALIZACAO-VOR/ILS/DME', 'CANALIZACAO-VOR/ILS/DME', regex=True
		)

		return df

	def _format(
		self,
		dfs: List,  # List with the individual API sources
	) -> pd.DataFrame:  # Processed DataFrame
		aero = dfs.pop()
		anatel = pd.concat(dfs, ignore_index=True)
		df = merge_on_frequency(anatel, aero)
		df = self.validate_coordinates(df)
		df = self._simplify_sources(df)
		df.sort_values(['Frequência', 'Latitude', 'Longitude'], ignore_index=True, inplace=True)
		return df.loc[:, self.columns]


# %%

In [ ]:
st = Estacoes(limit=100000)

`=======`

In [ ]:
# |export
class Estacoes(Base):
    """Classe auxiliar para agregar os dados originários da Anatel"""

    def __init__(
        self,
        sql_params: dict = SQLSERVER_PARAMS,
        mongo_uri: str = MONGO_URI,
        limit: int = 0,
        parallel: bool = True,
    ):
        self.sql_params = sql_params
        self.mongo_uri = mongo_uri
        self.limit = limit
        self.parallel = parallel
        self.init_data_sources()

    @property
    def columns(self):
        return COLS_SRD

    def build_from_sources(self) -> pd.DataFrame:
        return self._format([s.df() for s in self.sources.values()])

    @property
    def stem(self):
        return "anatel"

    @staticmethod
    def _update_source(class_instance):
        class_instance.update()
        class_instance.save()
        return class_instance

    def init_data_sources(self):
        self.sources = {
            "telecom": Telecom(self.mongo_uri, self.limit),
            "smp": SMP(self.mongo_uri, self.limit),
            "srd": SRD(self.mongo_uri),
            "stel": Stel(self.sql_params),
            "radcom": Radcom(self.sql_params),
            "aero": Aero(),
        }

    def extraction(self) -> L:
        if self.parallel:
            sources = parallel(
                Estacoes._update_source,
                self.sources.values(),
                n_workers=len(self.sources),
                progress=True,
            )
        else:
            sources = L(self._update_source(s) for s in self.sources.values())
        return sources.attrgot("df")

    @staticmethod
    def verify_shapefile_folder():
        # Convert the file paths to Path objects
        shapefile_path = Path(IBGE_POLIGONO)
        parent_folder = shapefile_path.parent
        parent_folder.mkdir(exist_ok=True, parents=True)
        zip_file_path = parent_folder.with_suffix(".zip")

        # Check if all required files exist
        required_files = L(".cpg", ".dbf", ".prj", ".shx").map(
            shapefile_path.with_suffix
        )
        if not all(required_files.map(Path.is_file)):
            # shutil.rmtree(str(shapefile_path.parent), ignore_errors=True)
            parent_folder.ls().map(Path.unlink)
            # Download and unzip the zipped folder
            urllib.request.urlretrieve(MALHA_IBGE, zip_file_path)
            with ZipFile(zip_file_path, "r") as zip_ref:
                zip_ref.extractall(parent_folder)
            zip_file_path.unlink()

    def fill_nan_coordinates(
        self,
        df: pd.DataFrame,  # DataFrame com os dados da Anatel
    ) -> pd.DataFrame:  # DataFrame com as coordenadas validadas na base do IBGE
        """Valida as coordenadas consultado a Base Corporativa do IBGE, excluindo o que já está no cache na versão anterior"""

        municipios = pd.read_csv(
            IBGE_MUNICIPIOS,
            usecols=["Código_Município", "Latitude", "Longitude"],
            dtype="string[pyarrow]",
            dtype_backend="pyarrow",
        )

        df = pd.merge(df, municipios, on="Código_Município", how="left", copy=False)

        null_coords = df.Latitude_x.isna() | df.Longitude_x.isna()

        df.loc[null_coords, ["Latitude_x", "Longitude_x"]] = df.loc[
            null_coords, ["Latitude_y", "Longitude_y"]
        ]

        log = """[("Colunas", ["Latitude", "Longitude"]),
		           ("Processamento", "Coordenadas Ausentes. Inserido coordenadas do Município")]"""
        df = self.register_log(df, log, null_coords)

        df.rename(
            columns={
                "Latitude_x": "Latitude",
                "Longitude_x": "Longitude",
                "Latitude_y": "Latitude_ibge",
                "Longitude_y": "Longitude_ibge",
            },
            inplace=True,
        )

        return df

    def intersect_coordinates_on_poligon(
        self, df: pd.DataFrame, check_municipio: bool = True
    ):
        for column in ["Latitude", "Longitude"]:
            df[column] = pd.to_numeric(df[column], errors="coerce").astype("float")
        regions = gpd.read_file(IBGE_POLIGONO)

        # Convert pandas dataframe to geopandas df with geometry point given coordinates
        gdf_points = gpd.GeoDataFrame(
            df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude)
        )

        # Set the same coordinate reference system (CRS) as the regions shapefile
        gdf_points.crs = regions.crs

        # Spatial join points to the regions
        gdf = gpd.sjoin(gdf_points, regions, how="inner", predicate="within")

        if check_municipio:
            # Check correctness of Coordinates
            check_coords = gdf.Código_Município != gdf.CD_MUN

            log = """[("Colunas", ["Código_Município", "Município", "UF"]),
				  	 ("Processamento", "Informações substituídas  pela localização correta das coordenadas.")		      
				  """
            self.register_log(gdf, log, check_coords)

            gdf.drop(["Código_Município", "Município", "UF"], axis=1, inplace=True)

        gdf.rename(
            columns={
                "CD_MUN": "Código_Município",
                "NM_MUN": "Município",
                "SIGLA_UF": "UF",
            },
            inplace=True,
        )

        return gdf

    def validate_coordinates(
        self, df: pd.DataFrame, check_municipio: bool = True
    ) -> pd.DataFrame:
        """
        Validates the coordinates in the given DataFrame.

        Args:
                df: The DataFrame containing the coordinates to be validated.
                check_municipio: A boolean indicating whether to check the municipality information (default: True).

        Returns:
                pd.DataFrame: The DataFrame with validated coordinates.

        Raises:
                None
        """
        self.verify_shapefile_folder()
        if check_municipio:
            df = self.fill_nan_coordinates(df)
        return self.intersect_coordinates_on_poligon(df, check_municipio)

    def _format(
        self,
        dfs: List,  # List with the individual API sources
    ) -> pd.DataFrame:  # Processed DataFrame
        aero = dfs.pop()
        anatel = pd.concat(dfs, ignore_index=True)
        df = self.validate_coordinates(merge_on_frequency(anatel, aero))
        df.sort_values(
            ["Frequência", "Latitude", "Longitude"], ignore_index=True, inplace=True
        )
        return df.loc[:, self.columns]

In [ ]:
#| eval: false
import os

In [ ]:
#|eval: fals
SQLSERVER_PARAMS = dict(
	driver='{ODBC Driver 17 for SQL Server}',
	server='ANATELBDRO05',
	database='SITARWEB',
	trusted_conn=False,
	mult_results=True,
	encrypt=False,
	username=os.environ['USERNAME'],
	password=os.environ['PASSWORD'],
	timeout=1000,
)


In [ ]:
st = Estacoes(sql_params=SQLSERVER_PARAMS, limit=100000, parallel=False)

`>>>>>>> 097aa98614b8d9e9f7c01af2511baaef3968e366`

In [ ]:
dfs = st.extraction()

ServerSelectionTimeoutError: anatelbdro06:27017: [Errno -2] Name or service not known, Timeout: 5.0s, Topology Description: <TopologyDescription id: 65521e9835f63e2badf88e13, topology_type: Unknown, servers: [<ServerDescription ('anatelbdro06', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('anatelbdro06:27017: [Errno -2] Name or service not known')>]>

`<<<<<<< HEAD`

`=======`

In [ ]:
aero = dfs.pop()
aero

,Frequência,Latitude,Longitude,Entidade,Fonte
0,0.1143,-32.340056,-54.223889,NDB - MELO,AISGEO
1,0.2000,-22.923333,-42.071500,NDB - CABO FRIO,AISGEO
2,0.2050,-5.386167,-35.531000,NDB - MAXARANGUAPE,AISGEO
3,0.2100,-19.561000,-46.964667,NDB - ARAXA,AISGEO
4,0.2250,-25.408667,-54.621667,NDB - ITAIPU,AISGEO
...,...,...,...,...,...
2990,995.0000,-23.631380,-46.652060,DME - AEROPORTO DE CONGONHAS 34X,AISGEO
2991,995.0000,-5.771808,-35.350192,DME - SAO GONCALO 34X,AISGEO
2992,995.0000,-9.336946,-54.952428,DME - CACHIMBO 34X,AISGEO
2993,995.0000,-9.509805,-35.786944,DME - IMC 34X,AISGEO


In [ ]:
aero.Fonte.value_counts()

Fonte
ICAO                                             1871
ICAO | AISWEB                                     265
AISGEO                                            230
ICAO-CANALIZACAO-VOR                              174
RADARES                                           154
AISWEB                                            103
AISWEB-CANALIZACAO-VOR                             78
ICAO-CANALIZACAO-VOR | AISWEB-CANALIZACAO-VOR      48
ICAO | AISGEO                                      46
REDEMET                                            26
Name: count, dtype: Int64

In [ ]:
anatel = pd.concat(dfs, ignore_index=True)
df = merge_on_frequency(anatel, aero)


AssertionError: O Agrupamento por colunas únicas não tem o comprimento esperado: 0!= 2110

In [ ]:
%debug]

> /home/rsilva/rfdatahub/extracao/format.py(198)merge_on_frequency()
    196 	assert (
    197                 len(df_both_left) == intersection_left
--> 198 	), f'O Agrupamento por colunas únicas não tem o comprimento esperado: {len(df_both_left)}!= {intersection_left}'
    199 
    200 	assert (



['Frequência', 'Entidade_x', 'Fistel', 'Número_Estação', 'Município', 'Código_Município', 'UF', 'Latitude_x', 'Longitude_x', 'Classe', 'Num_Serviço', 'Classe_Emissão', 'Largura_Emissão(kHz)', 'Validade_RF', 'Status', 'Fonte_x', 'Multiplicidade', 'Log', 'Altura_Estação', 'Potência_Transmissor(W)', 'Ganho_Antena(dBd)', 'Ângulo_Elevação_Antena', 'Azimute_Antena', 'Altura_Antena(m)', 'Atenuação_Linha(db/100m)', 'Perdas_Acessórias_Linha(db)', 'Padrão_Antena(dBd)']
['Frequência', 'Entidade_x', 'Fistel', 'Número_Estação', 'Município', 'Código_Município', 'UF', 'Latitude_x', 'Longitude_x', 'Classe', 'Num_Serviço', 'Classe_Emissão', 'Largura_Emissão(kHz)', 'Validade_RF', 'Status', 'Fonte_x', 'Multiplicidade', 'Log', 'Altura_Estação', 'Potência_Transmissor(W)', 'Ganho_Antena(dBd)', 'Ângulo_Elevação_Antena', 'Azimute_Antena', 'Altura_Antena(m)', 'Atenuação_Linha(db/100m)', 'Perdas_Acessórias_Linha(db)', 'Padrão_Antena(dBd)']


`>>>>>>> 097aa98614b8d9e9f7c01af2511baaef3968e366`